In [1]:
import random
import numpy as np
import scipy.io as sio

In [2]:
# R = number of ROIs
# L = integer with lag values to use
# n = number of time points
# S_g = number of subjects for each group
# S = total number of subjects
# coefs = number of VAR coefficients
# P = number of covariates
# G = number of groups

R = 10   
L = 1      
n = 200
S_g = [30,60]
S = sum(S_g)
coefs = R*R*L
P = 6
G = 2

In [3]:
###### generate data ######
max_x = 75
while max_x > 10:

    # generate groups
    groups = np.concatenate([np.repeat(0,S_g[0]),np.repeat(1,S_g[1])])

    # generate covariates
    COV_min = -0.99
    COV_max = 0.99

    COV = np.zeros((P,S))
    for g in range(G):
        COV[0,] = np.random.uniform(COV_min,COV_max,S)
        COV[1,] = np.random.uniform(COV_min,COV_max,S)
        COV[2,] = np.random.uniform(COV_min,COV_max,S)
        COV[3,] = np.random.uniform(COV_min,COV_max,S)
        COV[4,] = np.random.uniform(COV_min,COV_max,S)
        COV[5,] = np.random.binomial(1,0.5,S)

    # generate functions
    q_B_mean_true = []
    for g in range(G):
        q_B_mean_true += [np.zeros((R,R,S_g[g]))]

    relationships = np.zeros((coefs,P))
    true_group_network = np.zeros((R,R,G))
    true_covariate_associations = np.zeros((R,R,G,P))
    line = np.linspace(-1,1,200)
    true_functions = np.zeros((R,R,G,P))

    alphas = np.random.normal(0,0.05,(R*L,R,S))

    prob_of_association = 0.20

    for r0 in range(R*L):
        for r1 in range(R):
            covariate = np.random.randint(5)
            covariates = np.random.choice(5,2,replace = False)
            for g in range(G):
                rand_no_edge = np.random.binomial(1,prob_of_association,1)
                plus_minus = np.random.choice([-1,1])
                if r0 == r1 and rand_no_edge == 0:
                    alphas[r0,r1,groups == g] += plus_minus*0.15
                    q_B_mean_true[g][r0,r1,:] += plus_minus*0.15
                    true_group_network[r0,r1,g] = 1
                    true_functions[r0,r1,g,covariate] = plus_minus*1
                if np.absolute(r0 - r1) == 1 and rand_no_edge == 0:
                    alphas[r0,r1,groups == g] += plus_minus*COV[covariate,groups == g]*0.25
                    q_B_mean_true[g][r0,r1,:] += plus_minus*COV[covariate,groups == g]*0.25
                    true_group_network[r0,r1,g] = 1
                    true_functions[r0,r1,g,covariate] = plus_minus*2
                    true_covariate_associations[r0,r1,g,covariate] = 1
                if np.absolute(r0 - r1) == 2 and rand_no_edge == 0:
                    a = plus_minus*((1.0 + COV[covariate,groups == g])**0.40)
                    alphas[r0,r1,groups == g] += np.interp(a,(a.min(),a.max()),(-0.20,0.20))
                    q_B_mean_true[g][r0,r1,:] += np.interp(a,(a.min(),a.max()),(-0.20,0.20))
                    true_group_network[r0,r1,g] = 1
                    true_functions[r0,r1,g,covariate] = plus_minus*3
                    true_covariate_associations[r0,r1,g,covariate] = 1
                if np.absolute(r0 - r1) == 3 and rand_no_edge == 0:
                    a = plus_minus*(COV[covariate,groups == g]**2)
                    alphas[r0,r1,groups == g] += np.interp(a,(a.min(),a.max()),(-0.20,0.20))
                    q_B_mean_true[g][r0,r1,:] += np.interp(a,(a.min(),a.max()),(-0.20,0.2))
                    true_group_network[r0,r1,g] = 1
                    true_functions[r0,r1,g,covariate] = plus_minus*4
                    true_covariate_associations[r0,r1,g,covariate] = 1
                if np.absolute(r0 - r1) == 4 and rand_no_edge == 0:
                    alphas[r0,r1,groups == g] += plus_minus*np.sin(np.pi*COV[covariate,groups == g])*0.20
                    q_B_mean_true[g][r0,r1,:] += plus_minus*np.sin(np.pi*COV[covariate,groups == g])*0.20
                    true_group_network[r0,r1,g] = 1
                    true_functions[r0,r1,g,covariate] = plus_minus*5
                    true_covariate_associations[r0,r1,g,covariate] = 1
                if np.absolute(r0 - r1) == 5 and rand_no_edge == 0:
                    alphas[r0,r1,groups == g] += plus_minus*COV[5,groups == g]*0.20
                    q_B_mean_true[g][r0,r1,:] += plus_minus*COV[5,groups == g]*0.20
                    true_group_network[r0,r1,g] = 1
                    true_functions[r0,r1,g,covariate] = plus_minus*6
                    true_covariate_associations[r0,r1,g,5] = 1
                if np.absolute(r0 - r1) == 6 and rand_no_edge == 0:
                    alphas[r0,r1,groups == g] += (COV[covariates[0],groups == g])*0.3
                    alphas[r0,r1,groups == g] -= (COV[covariates[1],groups == g])*0.3
                    q_B_mean_true[g][r0,r1,:] += (COV[covariates[0],groups == g])*0.3 - (COV[covariates[1],groups == g])*0.3
                    true_group_network[r0,r1,g] = 1
                    true_functions[r0,r1,g,covariates] = 7
                    true_covariate_associations[r0,r1,g,covariates] = 1
                if np.absolute(r0 - r1) == 7 and rand_no_edge == 0:
                    a = ((1.0 + COV[covariate,groups == g])**0.70)
                    alphas[r0,r1,groups == g] += plus_minus*np.interp(a,(a.min(),a.max()),(-0.15,0.35))
                    q_B_mean_true[g][r0,r1,:] += plus_minus*np.interp(a,(a.min(),a.max()),(-0.15,0.35))
                    true_group_network[r0,r1,g] = 1
                    true_functions[r0,r1,g,covariate] = plus_minus*8
                    true_covariate_associations[r0,r1,g,covariate] = 1 

    # generate time series
    X = np.zeros([n,R,S])
    X[0,:,:] = np.random.normal(0,0.25,(R,S))

    for s in range(S):
            for i in range(n - 1):
                X[i + 1,:,s] = X[i,:,s].dot(alphas[:,:,s]) + np.random.normal(0,0.5,R)
            xx = X[:,:,s]

    max_x = np.absolute(X).max()
    
# save data
sio.savemat('data.mat', {'X':X,'True_network':true_group_network,'Covariates':COV, 'True_covariate_network':true_covariate_associations, 'True_functions':true_functions, 'True_subject_edges':alphas,'Groups':groups})